<a href="https://colab.research.google.com/github/KalpanaMehta/KalpanaMehta/blob/main/transfer_learning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
!pwd

/content


In [3]:
!mkdir deep_learning

In [4]:
%cd deep_learning/

/content/deep_learning


In [8]:
!mkdir images

In [11]:
%cd images

/content/deep_learning/images


In [12]:
from google.colab import files
upload = files.upload()

Saving tranfer_Image.jpeg to tranfer_Image.jpeg


In [6]:
from keras.applications import ResNet50
model = ResNet50()

102967424/102967424 [==============================] - 1s 0us/step


In [7]:
model.summary()

In [15]:
from keras.preprocessing.image import load_img
image = load_img('/content/deep_learning/images/tranfer_Image.jpeg',target_size=(224,224))


In [16]:
#converting pixels to numpy array

from keras.preprocessing.image import img_to_array
image = img_to_array(image)

In [17]:
import numpy as np
image = np.expand_dims(image,axis =0)

In [19]:
from keras.applications.vgg16 import preprocess_input
image = preprocess_input(image)

In [22]:
pred = model.predict(image)

1/1 [==============================] - 0s 203ms/step


In [23]:
from tensorflow.keras.applications.mobilenet import decode_predictions
pred_classes = decode_predictions(pred, top=5)
for i in pred_classes[0]:
    print(i)

35363/35363 [==============================] - 0s 0us/step
('n01614925', 'bald_eagle', 0.93017083)
('n01795545', 'black_grouse', 0.031570375)
('n01798484', 'prairie_chicken', 0.010568842)
('n02002724', 'black_stork', 0.008273316)
('n01608432', 'kite', 0.0068677864)


In [25]:
from keras.layers import Conv2D, UpSampling2D, Input
from keras.models import Sequential, Model
from keras.preprocessing.image import ImageDataGenerator, img_to_array, load_img
from skimage.color import rgb2lab, lab2rgb,gray2rgb
from skimage.transform import resize
from skimage.io import imsave
import numpy as np
import tensorflow as tf
import keras
import os

In [26]:
from keras.applications.vgg16 import VGG16
vgg_model = VGG16()

553467096/553467096 [==============================] - 8s 0us/step


In [27]:
newmodel = Sequential()
for i, layer in enumerate(vgg_model.layers):
    if i<19:          #Only up to 19th layer to include feature extraction only
      newmodel.add(layer)
newmodel.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 block1_conv1 (Conv2D)       (None, 224, 224, 64)      1792      
                                                                 
 block1_conv2 (Conv2D)       (None, 224, 224, 64)      36928     
                                                                 
 block1_pool (MaxPooling2D)  (None, 112, 112, 64)      0         
                                                                 
 block2_conv1 (Conv2D)       (None, 112, 112, 128)     73856     
                                                                 
 block2_conv2 (Conv2D)       (None, 112, 112, 128)     147584    
                                                                 
 block2_pool (MaxPooling2D)  (None, 56, 56, 128)       0         
                                                                 
 block3_conv1 (Conv2D)       (None, 56, 56, 256)       2

In [28]:

for layer in newmodel.layers:
  layer.trainable=False

In [ ]:
path = 'images/colorization/'
#Normalize images - divide by 255
train_datagen = ImageDataGenerator(rescale=1. / 255)